<a href="https://colab.research.google.com/github/hectorhung142857/-/blob/main/%E3%80%8C%E7%88%AC%E8%9F%B2%EF%BC%8B%E5%88%86%E9%A1%9E%2B%E6%83%85%E7%B7%92_ipynb%E3%80%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
from joblib import load
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin  # 用來處理相對URL
import numpy as np
import sys

# 載入模型
model = load("/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/final_model.joblib")
vectorizer = load("/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/final_vectorizer.joblib")
stemmer = PorterStemmer()

# 前處理函數
def preprocess_text(text):
    text = re.sub(r'[,-;\\()<>_\'\"]', '', text)  # 去除標點符號
    text = text.lower()  # 轉小寫
    text = re.sub(r'^\d{1,2} [A-Za-z]+ \d{4}', '', text).strip()  # 去除日期
    text = text[4:] # 去除月份
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

#------------爬取文章-------------

# 定義抓取新聞的基礎 URL 和搜尋關鍵字

search_url = "https://www.bbc.com/search?q="
n=3
keyword = np.empty(n,dtype='<U20')
keyword[0] = ""
keyword[1] = ""
keyword[2] = ""

# 輸入字串
user_input = input("請輸入一至三個關鍵字，以空格分開：")
# 分割字串
words = user_input.split(" ")
num_words = len(words)
if num_words >=1 and num_words <=3:
    for i in range(len(words)):
        keyword[i] = words[i]
else:
    print('輸入格式錯誤')
    sys.exit()

if keyword[0] == "" and keyword[1] == "" and keyword[2] == "":
    print('輸入格式錯誤')
    sys.exit()

keyword_num = 3
while(keyword_num > 0):
    keyword_num-=1
    i=keyword_num
    if not keyword[i]:
        continue
    search_url = f'{search_url}{keyword[i]}+'

url = search_url.rstrip("+")

print(url)

# 儲存新聞數據的列表
data = []

# 控制抓取的頁數
max_pages = 100 # 只抓取前 n+1 頁
page_num = 0

# 抓取的文章最大數
max_article = 30
article_num = 0

while page_num <= max_pages and article_num <= max_article:
    # 構建當前頁的 URL
    page_url = f'{url}&page={page_num}'

    # 發送 GET 請求
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 找到所有新聞鏈接
    articles = soup.find_all('a', href=True)

    # 遍歷每個鏈接
    for article in articles:
        article_text = article.get_text().lower()  # 轉換為小寫，忽略大小寫
        article_url = urljoin('https://www.bbc.com', article['href'])  # urljoin 會自動處理相對路徑

        if '/news/article' in article_url or '/news/business' in article_url:

            article_response = requests.get(article_url)
            article_soup = BeautifulSoup(article_response.content, 'html.parser')

            # 嘗試找到文章的內文，通常是 <p> 標籤中的內容
            paragraphs = article_soup.find_all('p')

            # 提取並打印文章內文（合併所有段落）
            article_content = '\n'.join([para.get_text() for para in paragraphs])

            data.append({
                'URL': article_url,
                'Title': article.get_text(),
                'Content': article_content,    #[:2000]  # 只儲存內文的前 2000 字
                'Type': ''
            })
            article_num+=1
            print('article :',article_num)
    page_num+=1
    print('page :',page_num)

# 將數據保存到 Excel 文件中
df = pd.DataFrame(data)

# 指定輸出的 Excel 文件路徑
output_article = '/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/article.xlsx'

# 將 DataFrame 輸出到 Excel
df.to_excel(output_article, index=False, engine='openpyxl')
print(f"Data has been successfully saved to {output_article}")

#-----------爬取成功--------------


# 讀取 CSV
input_file = "/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/article.xlsx"
data = pd.read_excel(input_file)
print(data.columns.tolist())

# 前處理
data['Processed Text'] = data['Content'].apply(preprocess_text)

# 新增自動生成的 id 欄位
data['id'] = range(1, len(data) + 1)

# 篩選需要的欄位
result_df = data[['id', 'Processed Text']]

# 顯示結果
print(result_df.head())


# 轉成 TF-IDF
tf_idf_vectors = vectorizer.transform(data['Processed Text'])

# 分類
data['Predicted Category'] = model.predict(tf_idf_vectors)

# 整理分類結果
categories = ['Alternative', 'Equity', 'fix income', 'Cash']
category_dict = {category: [] for category in categories}
for idx, row in data.iterrows():
    category_dict[row['Predicted Category']].append(row['id'])

# 補空值，避免 DataFrame 格式錯誤
max_length = max(len(ids) for ids in category_dict.values())
formatted_data = {category: ids + [""] * (max_length - len(ids)) for category, ids in category_dict.items()}

# 轉為 DataFrame 格式
output_df = pd.DataFrame(formatted_data)

# 輸出結果到 CSV
output_file = "/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/classified_articles_output.csv"
output_df.to_csv(output_file, index=False)

print(f"分類完成，結果已保存到 {output_file}")
print(output_df)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator GridSearchCV from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator Tfi

請輸入一至三個關鍵字，以空格分開：USA election Trump
https://www.bbc.com/search?q=Trump+election+USA
article : 1
article : 2
article : 3
article : 4
article : 5
article : 6
article : 7
article : 8
page : 1
article : 9
article : 10
article : 11
article : 12
article : 13
article : 14
article : 15
article : 16
page : 2
article : 17
article : 18
article : 19
article : 20
article : 21
article : 22
article : 23
page : 3
article : 24
article : 25
article : 26
article : 27
article : 28
article : 29
article : 30
page : 4
article : 31
article : 32
article : 33
page : 5
Data has been successfully saved to /content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/article.xlsx
['URL', 'Title', 'Content', 'Type']
   id                                     Processed Text
0   1  resid joe biden ha said he think he would have...
1   2  re polit donald trump built a dynasti with a m...
2   3  keir starmer will visit washington for talk wi...
3   4  insan ! were head for a gener elect the countr...
4   5  highprofil nigerian cleric

In [ ]:
情緒分析

In [ ]:
print(result_df.head())

   id                                     Processed Text
0   1  resid joe biden ha said he think he would have...
1   2  re polit donald trump built a dynasti with a m...
2   3  keir starmer will visit washington for talk wi...
3   4  insan ! were head for a gener elect the countr...
4   5  highprofil nigerian cleric took part in event ...


In [ ]:
'''
依據文章分類抓取文章['id', 'Processed Text'
'''
import pandas as pd
def loading_news (nwes_id_df,id_asset_df):
  asset_news_dict = {"Equity":[],"fix income":[],"Alternative":[],"Cash":[]}
  id_asset_dict = id_asset_df.to_dict(orient="list")
  news_id_dict = dict(zip(nwes_id_df["id"], nwes_id_df["Processed Text"]))
  print(id_asset_dict)
  print(news_id_dict)
  for asset in ["Equity","fix income","Alternative","Cash"]:
    for news_id in id_asset_dict[asset]:
      #news_id = str(news_id)
      if news_id not in news_id_dict:
        continue
      news = news_id_dict[news_id]
      asset_news_dict[asset].append(news)
  return asset_news_dict
nwes_id_df = result_df
id_asset_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/文字探勘期末專案/classified_articles_output.csv")
training_dict = loading_news(nwes_id_df,id_asset_df)
print(training_dict)

{'Alternative': [8.0, 24.0, 29.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'Equity': [1, 2, 5, 6, 9, 11, 14, 15, 16, 18, 21, 22, 23, 28, 32, 33], 'fix income': [3.0, 4.0, 7.0, 10.0, 12.0, 13.0, 17.0, 19.0, 20.0, 25.0, 26.0, 27.0, 30.0, 31.0, nan, nan], 'Cash': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}
{1: 'resid joe biden ha said he think he would have defeat donald trump and won reelect in novemb speak to usa today in an exclus interview biden did howev add that he wa unsur if he would have had the stamina for anoth fouryear term so far so good the yearold said but who know what im go to be when im year old ? in the widerang interview with susan page biden also said he wa still consid preemptiv pardon for foe of donald trump includ former republican congresswoman liz cheney and former senior health offici dr anthoni fauci in the interview publish on wednesday biden said he had been veri straightforward with trump about t

In [ ]:
"""
過FinBERT模型進行情緒分析
"""
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 檢查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model directly
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
model = model.to(device)
import torch
import scipy.special
import pandas as pd

def analyze_finbert_sentiment(news_data, model, tokenizer, device='cpu'):
    """
    使用 FinBERT 模型對一批新聞文本進行情感分析，回傳結果 DataFrame。

    參數：
      news_data (list of str): 新聞文本清單，每個元素為一篇新聞的文字。
      model: 已載入並可用的 FinBERT (或相容) 模型。
      tokenizer: 與模型搭配的 tokenizer。
      device (str or torch.device): 'cpu' 或 'cuda'，預設為 'cpu'。

    回傳：
      result_df (pd.DataFrame): 包含每篇新聞的負面、中立、正面分數與預測標籤。
    """
    # 確保模型運行在指定的 device
    model.to(device)
    model.eval()

    # 初始化儲存列表
    results = []

    # 定義 tokenizer 的參數
    tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}

    # 遍歷新聞文本
    for x in news_data:
        with torch.no_grad():
            # 將新聞文本進行編碼並移動到 GPU
            input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs).to(device)

            # 模型推理
            logits = model(**input_sequence).logits.cpu()  # 將輸出轉回 CPU
            print(f"logits: {logits}")
            # 計算 softmax 機率
            probabilities = scipy.special.softmax(logits.numpy().squeeze())

            # 建立情感標籤對應
            sentiment_labels = model.config.id2label.values()
            scores = {label: prob for label, prob in zip(sentiment_labels, probabilities)}

            # 提取最高概率的情感
            predicted_label = max(scores, key=scores.get)

            # 保存結果
            results.append({
                "Negative_Score": scores["negative"],
                "Neutral_Score": scores["neutral"],
                "Positive_Score": scores["positive"],
                "Predicted_Label": predicted_label
            })

    # 將結果轉為 DataFrame
    result_df = pd.DataFrame(results)
    return result_df


# 分析並保存 Equity 的情感結果
equity = analyze_finbert_sentiment(training_dict['Equity'], model, tokenizer)
equity.to_csv("equity_sentiment.csv", index=False)
print("Equity sentiment saved to 'equity_sentiment.csv'.")
print(equity.head())

# 分析並保存 Fixed Income 的情感結果
fixed_income = analyze_finbert_sentiment(training_dict['fix income'], model, tokenizer)
fixed_income.to_csv("fixed_income_sentiment.csv", index=False)
print("Fixed Income sentiment saved to 'fixed_income_sentiment.csv'.")

# 分析並保存 Alternatives 的情感結果
alternatives = analyze_finbert_sentiment(training_dict['Alternative'], model, tokenizer)
alternatives.to_csv("alternatives_sentiment.csv", index=False)
print("Alternatives sentiment saved to 'alternatives_sentiment.csv'.")

# 分析並保存 Cash 的情感結果
cash = analyze_finbert_sentiment(training_dict['Cash'], model, tokenizer)
cash.to_csv("cash_sentiment.csv", index=False)
print("Cash sentiment saved to 'cash_sentiment.csv'.")

Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

logits: tensor([[-0.9890, -0.7542,  2.1427]])
logits: tensor([[-0.9801, -0.8103,  2.2813]])
logits: tensor([[-0.7675, -1.2782,  2.2600]])
logits: tensor([[-0.7453, -1.3045,  2.2188]])
logits: tensor([[-1.4998,  1.0388,  1.4632]])
logits: tensor([[-0.8815, -1.0258,  2.2654]])
logits: tensor([[-0.5859, -1.0753,  2.0948]])
logits: tensor([[-0.6835, -1.1664,  2.2205]])
logits: tensor([[-0.0190, -1.8688,  1.9968]])
logits: tensor([[-0.9297, -1.0375,  2.2933]])
logits: tensor([[-0.8619, -0.7724,  2.2284]])
logits: tensor([[-0.8570, -1.0280,  2.2466]])
logits: tensor([[-1.3987,  0.1265,  2.1110]])
logits: tensor([[-0.4896, -1.4675,  2.1978]])
logits: tensor([[-0.8571, -1.0992,  2.3017]])
logits: tensor([[-1.0243, -0.4470,  2.1311]])
Equity sentiment saved to 'equity_sentiment.csv'.
   Negative_Score  Neutral_Score  Positive_Score Predicted_Label
0        0.050231       0.910049        0.039720         neutral
1        0.041917       0.922711        0.035372         neutral
2        0.026973  

視覺化


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 假設讀取的 CSV 檔案列表
file_list = ["equity_sentiment.csv", "fixed_income_sentiment.csv", "alternatives_sentiment.csv", "cash_sentiment.csv"]
assets = ["Equity", "Fixed Income", "Alternative", "Cash"]

# 初始化結果 DataFrame
result_df = pd.DataFrame(columns=["Asset", "Positive", "Neutral", "Negative", "Article Count", "Average Total Impact"])

# 讀取各 CSV 檔案並匯總
for file, asset in zip(file_list, assets):
    df = pd.read_csv(file)
    positive_sum = df["Positive_Score"].sum()
    neutral_sum = df["Neutral_Score"].sum()
    negative_sum = df["Negative_Score"].sum()
    article_count = len(df)  # 記錄文章篇數
    print(f"Asset: {asset}")
    print(f"Positive Sum: {positive_sum}")
    print(f"Neutral Sum: {neutral_sum}")
    print(f"Negative Sum: {negative_sum}")
    print(f"Article Count: {article_count}")
    # 計算新的平均總影響（每篇文章 max(Positive, Negative) 的平均值）
    max_impact_average = df.apply(
        lambda x: max(x["Positive_Score"], x["Negative_Score"]), axis=1
    ).mean()

    # 匯總結果
    result_df = pd.concat([result_df, pd.DataFrame([{
        "Asset": asset,
        "Positive": positive_sum,
        "Neutral": neutral_sum,
        "Negative": negative_sum,
        "Article Count": article_count,
        "Average Total Impact": max_impact_average
    }])], ignore_index=True)

# 排序結果（以新的平均總影響排序）
sorted_avg_total = result_df.sort_values(by="Average Total Impact", ascending=False)


##############
for file, asset in zip(file_list, assets):
    df = pd.read_csv(file)
    print(f"Sentiment counts for {asset}:")
    print(df['Predicted_Label'].value_counts())
#####################


# 創建圖表
fig1, axes = plt.subplots(2, 2, figsize=(16, 14))
fig1.canvas.manager.set_window_title("Sentiment Analysis Results")
axes = axes.flatten()
print(df)
# 圓餅圖：每個資產的情感比例
for i, (file, asset) in enumerate(zip(file_list, assets)):
    df = pd.read_csv(file)

    # 使用 y_pred 欄位統計情感分布
    sentiment_counts = df['Predicted_Label'].value_counts().sort_index()
    negative = sentiment_counts.get(0, 0)  # 負向（y_pred = 0）
    neutral = sentiment_counts.get(1, 0)   # 中立（y_pred = 1）
    positive = sentiment_counts.get(2, 0) # 正向（y_pred = 2）

    # 繪製圓餅圖
    axes[i].pie(
        [negative, neutral, positive],
        labels=["Negative", "Neutral", "Positive"],
        autopct="%1.1f%%",
        colors=["#66c2a5", "#fc8d62", "#8da0cb"]
    )
    axes[i].set_title(f"Sentiment Distribution for {asset}", fontsize=14)

# 平均總影響圖表
fig2, ax = plt.subplots(figsize=(10, 6))
fig2.canvas.manager.set_window_title("Average Total Impact Analysis")

# 繪製平均總影響圖表
ax.bar(sorted_avg_total["Asset"], sorted_avg_total["Average Total Impact"], color="skyblue")
ax.set_title("Average Total Impact per Article", fontsize=14)
ax.set_ylabel("Average Total Impact")
ax.set_xlabel("Assets")

# 顯示圖表
plt.tight_layout()
plt.show()


Asset: Equity
Positive Sum: 0.7507762229999999
Neutral Sum: 14.24421767
Negative Sum: 1.005006277
Article Count: 16
Asset: Fixed Income
Positive Sum: 0.6201227090000001
Neutral Sum: 12.155259929999998
Negative Sum: 1.224617382
Article Count: 14
Asset: Alternative
Positive Sum: 0.121280467
Neutral Sum: 2.7120318
Negative Sum: 0.16668771300000002
Article Count: 3


<ipython-input-8-20db83f7bbb6>:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame([{


EmptyDataError: No columns to parse from file